In [1]:
from nonasymptotic.envs import StraightLine
from nonasymptotic.prm import SimplePRM

import numpy as np

In [2]:
seed = 1
prm_env_2d = StraightLine(dim=2, delta_clearance=0.5, seed=seed)
prm = SimplePRM(0.10, prm_env_2d.is_motion_valid, prm_env_2d.sample_from_env, prm_env_2d.distance_to_path, seed=seed)
prm.grow_to_n_samples(1000)

# plt.figure()
# for u, v in prm.g_prm.iterEdges():
#     coords_u = prm.samples[u]
#     coords_v = prm.samples[v]
# 
#     plt.plot([coords_u[0], coords_v[0]], [coords_u[1], coords_v[1]], 'ro-')
# 
# plt.show()

# some other computations to help other visual tests later on:
print('N Components: %i' % len(prm.g_cc.getComponents()))
print('N nodes: %i' % prm.num_vertices())
print('N edges: %i' % prm.num_edges())


N Components: 1
N nodes: 1000
N edges: 8212


In [3]:
# do some connection queries with some existing points
s = np.array([0., 0.])
g = np.array([1., 0.])

sol_dist, sol_path = prm.query_best_solution(s, g)
sol_path = np.concatenate([[s], sol_path, [g]])

# plt.figure()
# 
# # plot the existing prm
# for u, v in prm.g_prm.iterEdges():
#     coords_u = prm.samples[u]
#     coords_v = prm.samples[v]
# 
#     plt.plot([coords_u[0], coords_v[0]], [coords_u[1], coords_v[1]], 'ro-')
# 
# for i in range(sol_path.shape[0] - 1):
#     u = sol_path[i]
#     v = sol_path[i + 1]
#     plt.plot([u[0], v[0]], [u[1], v[1]], 'bo-')
# 
# plt.plot()
print('N nodes: %i' % prm.num_vertices())
print('N edges: %i' % prm.num_edges())


N nodes: 1000
N edges: 8212


In [4]:
pairs, _, _ = prm.query_all_graph_connections(s, g)
# plt.figure()

# plot the existing prm
# for u, v in prm.g_prm.iterEdges():
#     coords_u = prm.samples[u]
#     coords_v = prm.samples[v]
# 
#     plt.plot([coords_u[0], coords_v[0]], [coords_u[1], coords_v[1]], 'ro-')
# 
# for u, v in pairs:
#     plt.plot([u[0], v[0]], [u[1], v[1]], 'bo-')
# 
# plt.show()


In [28]:
import time

times = []
for _ in range(100):
    t_start = time.process_time()
    prm_env_2d.is_prm_epsilon_delta_complete(prm, 0.28, n_samples_per_check=100, timeout=60.0, timeout_area_tol=1e-6, vis=True)
    t_end = time.process_time()
    times.append(t_end - t_start)
    
print('mean: %f, std: %f' % (np.mean(times), np.std(times)))

covered!
covered!
covered!
covered!
covered!
covered!
covered!


KeyboardInterrupt: 

In [23]:
start = np.array([0.5034941, 0.0])
goal = np.array([0.6034941, 0.0])

dist, path = prm.query_best_solution(start, goal)
print(dist - (1 + 0.256) * (goal[0] - start[0]))

# sol pair is (789, 789); [0.54648089 0.03865867]

print(np.linalg.norm(path[0] - path[0]))
sol_pts, sol_dists, sol_ids = prm.query_all_graph_connections(start, goal)

start_norms = np.linalg.norm(prm.samples - start, axis=1)
start_selector = start_norms <= 0.1
all_neighbors_start = np.arange(1000)[start_selector]
start_sort = np.argsort(start_norms[start_selector])

goal_norms = np.linalg.norm(prm.samples - goal, axis=-1)
goal_selector = goal_norms <= 0.1
all_neighbors_goal = np.arange(1000)[goal_selector]
goal_sort = np.argsort(goal_norms[goal_selector])
 

5.475329756740588e-05
0.0


In [7]:
prm.k_neighbors

32

In [12]:
print(prm.nn_index.query([start], k=15)[0][0])
print(all_neighbors_start[start_sort])
print(prm.nn_index.query([start], k=15)[1][0])
print(np.sort(start_norms[start_selector]))

print(prm.nn_index.query([goal], k=19)[0][0])
print(all_neighbors_goal[goal_sort])
print(prm.nn_index.query([goal], k=19)[1][0])
print(np.sort(goal_norms[goal_selector]))

np.linalg.norm(prm.samples[789] - start) + np.linalg.norm(prm.samples[789] - goal) - 1.20 * (goal[0] - start[0])

[171 897 944 789 172 581 282 841 315 477 614 501 261 494 729]
[171 897 944 789 172 581 282 841 315 477 614 501 261 494]
[0.02752812 0.03136257 0.0523346  0.0572447  0.06185896 0.06200621
 0.06251651 0.07732189 0.07831659 0.08200165 0.08363293 0.08393107
 0.09146561 0.09627791 0.10210367]
[0.02752812 0.03136257 0.0523346  0.05724469 0.06185896 0.06200621
 0.06251653 0.07732191 0.07831661 0.08200166 0.08363293 0.08393108
 0.09146563 0.09627791]
[150 841 755 696 282 315 789 261 186 534 262 571 477 881 897 463 665 729
 586]
[150 841 755 696 282 315 789 261 186 534 262 571 477 881]
[0.00383368 0.05566616 0.05584162 0.06454111 0.07452732 0.07526896
 0.07531777 0.07597972 0.07638264 0.0764377  0.08494437 0.08499358
 0.08618211 0.09057384 0.10130286 0.1100651  0.1106575  0.11180214
 0.11208599]
[0.00383369 0.05566616 0.0558416  0.06454111 0.07452732 0.07526896
 0.07531779 0.07597971 0.07638265 0.0764377  0.08494433 0.08499358
 0.08618211 0.09057384]


0.0043319162935468725